#### Simulation generation

In [ ]:
import numpy as np
import nibabel as nb
import os
from nibabel.testing import data_path

def no_transform(dirname):
    if not os.path.exists(dirname):
        os.makedirs(dirname)
        
    brain = np.random.randint(low=50, high=60, size=20*20, dtype=np.int16).reshape((20,20))
    #brain = np.arange(50*50*50, dtype=np.int16).reshape((50,50,50))
    identity = np.diag([1,1,1,1])
    brainimg = nb.Nifti1Image(brain, affine=identity)

    nb.save(brainimg, dirname + "/inp.nii.gz")
    nb.save(brainimg, dirname + "/ref.nii.gz")
    np.savetxt(dirname + "/correctaffine.txt", identity)

def translation(dirname):
    if not os.path.exists(dirname):
        os.makedirs(dirname)

    #brain = np.random.randint(low=50, high=60, size=50*50*50, dtype=np.int16).reshape((50,50,50))
    #brain = 50*np.ones((100,100), dtype=np.int16)
    brain = np.random.randint(low=30,high=70,size=100*100, dtype=np.int16).reshape((100,100))
    displacement = np.random.randint(low=20, high=40, size=1, dtype=np.int16)
    displacement = displacement[0]
    #zeros = np.zeros((displacement,50,50),dtype=np.int16)
    zeros = np.zeros((displacement,100),dtype=np.int16)
    identity = np.diag([1,1,1,1])

    brainmod = np.vstack((zeros, brain))
    brainimg = nb.Nifti1Image(brainmod, affine=identity)
    nb.save(brainimg, dirname + "/inp.nii.gz")

    brainmod = np.vstack((brain, zeros))
    brainimg = nb.Nifti1Image(brainmod, affine=identity)
    nb.save(brainimg, dirname + "/ref.nii.gz")

    identity[0,3] = displacement
    np.savetxt(dirname + "/correctaffine.txt", identity)

def scaling(dirname):
    if not os.path.exists(dirname):
        os.makedirs(dirname)

    brain = 50*np.ones((50,50,50), dtype=np.int16)
    #brain = 50*np.ones((50,50), dtype=np.int16)
    identity = np.diag([1,1,1,1])
    factor = np.random.randint(low=10, high=40, size=1, dtype=np.int16)
    f = factor[0]

    brainmod = np.lib.pad(brain, ((f,f),(f,f),(f,f)), 'constant', constant_values=(0))
    #brainmod = np.lib.pad(brain, ((f,f),(f,f)), 'constant', constant_values=(0))
    brainimg = nb.Nifti1Image(brainmod, affine=identity)
    nb.save(brainimg, dirname + "/inp.nii.gz")

    f=f-1
    brainmod = np.lib.pad(brain, ((f,f),(f,f),(f,f)), 'constant', constant_values=(50))
    #brainmod = np.lib.pad(brain, ((f,f),(f,f)), 'constant', constant_values=(50))
    brainmod = np.lib.pad(brainmod, ((1,1),(1,1),(1,1)), 'constant', constant_values=(0))
    #brainmod = np.lib.pad(brainmod, ((1,1),(1,1)), 'constant', constant_values=(0))
    brainimg = nb.Nifti1Image(brainmod, affine=identity)
    nb.save(brainimg, dirname + "/ref.nii.gz")

def rotation(dirname):
    if not os.path.exists(dirname):
        os.makedirs(dirname)

    identity = np.diag([1,1,1,1])
    brain = np.arange(50*50, dtype=np.int16).reshape((50,50))
    brainimg = nb.Nifti1Image(brain, affine=identity)
    nb.save(brainimg, dirname + "/inp.nii.gz")

    numrots = np.random.randint(low=1,high=4,size=1, dtype=np.int16)
    numrots = numrots[0]
    brainmod = np.rot90(brain, numrots)
    brainimg = nb.Nifti1Image(brainmod, affine=identity)
    nb.save(brainimg, dirname + "/ref.nii.gz")

    if (numrots == 1):
        identity[0,0] = 0
        identity[1,1] = 0
        identity[0,1] = 1
        identity[1,0] = -1
        identity[1,3] = 49
    if (numrots == 2):
        identity[0,0] = -1
        identity[1,1] = -1
        identity[0,1] = 0
        identity[1,0] = 0
        identity[1,3] = 49
        identity[0,3] = 49
    if (numrots == 3):
        identity[0,0] = 0
        identity[1,1] = 0
        identity[0,1] = -1
        identity[1,0] = 1
        identity[0,3] = 49
    np.savetxt(dirname + "/correctaffine.txt", identity)

def non_linear(dirname):
    if not os.path.exists(dirname):
        os.makedirs(dirname)

    identity = np.diag([1,1,1,1])
    #brain = np.arange(50*50, dtype=np.int16).reshape((50,50))
    brain = np.random.randint(low=10, high=40, size=50*50, dtype=np.int16).reshape((50,50))
    brainimg = nb.Nifti1Image(brain, affine=identity)
    nb.save(brainimg, dirname + "/inp.nii.gz")

    a = np.random.randint(low=10, high=40, size=1, dtype=np.int16)[0]
    b = np.random.randint(low=10, high=40, size=1, dtype=np.int16)[0]
    n = 50
    r = np.random.randint(low=10, high=40, size=1, dtype=np.int16)[0]
    y,x = np.ogrid[-a:n-a, -b:n-b]
    mask = x*x + y*y <= r*r

    brain[mask] = 0
    brainimg = nb.Nifti1Image(brain, affine=identity)
    nb.save(brainimg, dirname + "/ref.nii.gz")

if __name__ == '__main__':
    #rotation("1")
    
    for i in range(3):
        no_transform(str(i))
    for i in range(3,13):
        translation(str(i))
    for i in range(13,23):
        scaling(str(i))
    for i in range(23,26):
        rotation(str(i))
    for i in range(26,36):
        non_linear(str(i))


Each function above generates a simulation of two "brain" files: an input and a reference. Each simulation represents one type of linear transformation, meaning the input data should be some linear transformation of the reference data. 

The no_transformation simulation should produce two identical square matrices of random numbers. 

The translation simulation should produce an input square matrix of random intensities, and the reference matrix should look like a right-shifted version of the input.

The scaling simulation should produce a small input square matrix of equal intensity, and the reference matrix should be a bigger square matrix with the same intensity.

The rotation simulation should produce an input square matrix of uniformly increasing intensity, and the reference matrix should be a rotated version of the input.

The non_linear simulation should produce an input square matrix of random intensities, and the reference matrix should be the same as the input except with a circular chunk cut out, making the reference a non-linear transformation of the input.

Below are some of the simulation data produced. The left image in each pair shows the input and the right image shows the reference.

![no transformation](flirt_pics/none0.jpg)
![no transformation](flirt_pics/none1.jpg)
![no transformation](flirt_pics/none2.jpg)
![translation](flirt_pics/trans3.jpg)
![translation](flirt_pics/trans6.jpg)
![translation](flirt_pics/trans10.jpg)
![scaling](flirt_pics/scale13.jpg)
![scaling](flirt_pics/scale15.jpg)
![scaling](flirt_pics/scale17.jpg)
![rotation](flirt_pics/rot23.jpg)
![rotation](flirt_pics/rot24.jpg)
![non linear](flirt_pics/nlin26.jpg)
![non linear](flirt_pics/nlin28.jpg)
![non linear](flirt_pics/nlin30.jpg)

As can be seen above, the simulation data looks as expected.

#### Pseudocode

Linear registration works through the following general steps:

1) Initial search: perform a cursory pass through some transformation combinations to find one that results in a cost function value believed to be close to optimal.

2) While possible, use gradient descent to find a new combinations of transformations with a cost value closer to optimal.

3) Terminate when local minimum for cost function is reached.

The algorithm code is not provided because it is an external tool.

The FLIRT algorithm should perform very well on the "easy" simulations because they are all simple linear transformations, which is what FLIRT is designed for. The error should be very low. However, the algorithm should not perform well on the "hard" simulations because they are non-linear transformations, which is absolutely not what FLIRT is designed for. The error should be very high.

Below we see the results from performing the algorithm on the simulation data generated above. Each image represents the "registered" data, which should match the "reference" images from above.

![flirted](flirt_pics/flirted0.jpg)
![flirted](flirt_pics/flirted1.jpg)
![flirted](flirt_pics/flirted2.jpg)
![flirted](flirt_pics/flirted3.jpg)
![flirted](flirt_pics/flirted6.jpg)
![flirted](flirt_pics/flirted10.jpg)
![flirted](flirt_pics/flirted13.jpg)
![flirted](flirt_pics/flirted15.jpg)
![flirted](flirt_pics/flirted17.jpg)
![flirted](flirt_pics/flirted23.jpg)
![flirted](flirt_pics/flirted24.jpg)
![flirted](flirt_pics/flirted26.jpg)
![flirted](flirt_pics/flirted28.jpg)
![flirted](flirt_pics/flirted30.jpg)

For the linear transformations, we will use the following quantitative metric: sum of elements in difference matrix (between correct and registered affine matrix). Basically, we will take the difference between the correct affine matrix and the output of the linear registration, and then the sum of the elements in this matrix will be our error. 

For the non-linear transformations, we will use the following quantitative metric: we will compare the reference and registered data by taking the difference between the two matrices and summing the elements in the difference matrix. This will give us the number of pixels that were mis-registered.

Code below:

In [ ]:
import numpy as np
import nibabel as nb
import os

if __name__ == '__main__':
    errors = []
    for i in range(26):
        dirname = str(i)
        correct = np.loadtxt(dirname + "/correctaffine.txt")
        estimate = np.loadtxt(dirname + "/flirted.mat")
        error = np.sum(np.absolute(correct - estimate))
        errors.append(error)

    for i in range(26,36):
        flirted = nb.load(str(i) + "/flirted.nii.gz").get_data()
        correct = nb.load(str(i) + "/ref.nii.gz").get_data()

        flirted[flirted > 1] = 1
        correct[correct > 1] = 1
        error = np.sum(np.absolute(correct - flirted))
        errors.append(error)
    print errors


[0.0, 0.0, 0.0, 1.550825195960003, 3.422254708859532, 11.980898569719997, 7.9705488077200028, 7.9201462880400024, 7.9358725125999987, 17.886743152900003, 1.5286875421199995, 1.033651326572408, 7.9116499097000021, 2.818713867125501, 3.187876099440456, 1.478484436311636, 2.833005868538394, 1.4267679668403832, 3.612968803025274, 2.6988117453018945, 1.7665954798872798, 3.830883126957, 3.231849615262577, 3.5460977669955334, 0.74573975247713831, 0.74573975247713831, 2255, 2172, 1009, 317, 1944, 1802, 2440, 1696, 1007, 2163]

The numbers above represent the error from each simulation trial. We will visualize population performance by plotting the error from each trial in a scatter plot. We will compute the overall population performance by taking the average error of all the trials.

In [ ]:
x = range(0,3)
plt.scatter(x, errors[0:3], c='b', s=40, label='no transform')
x = range(23,26)
plt.scatter(x, errors[23:26], c='r', s=40, label='rotation')
x = range(3,13)
plt.scatter(x, errors[3:13], c='g', s=40, label='translation')
x = range(13,23)
plt.scatter(x, errors[13:23], c='y', s=40, label='scaling')
plt.legend(loc='upper right')
plt.title('Errors')
plt.savefig("errors1.png")
plt.cla()

x = range(0,10)
plt.scatter(x, errors[26:], s=40, label='non linear')
plt.legend(loc='upper right')
plt.title('Errors')
plt.savefig("errors2.png")
plt.cla()

![error](flirt_pics/errors1.jpg)
![error](flirt_pics/errors2.jpg)

In [ ]:
noneavg = np.mean(errors[0:3])
transavg = np.mean(errors[3:13])
scaleavg = np.mean(errors[13:23])
rotavg = np.mean(errors[23:26])
nlinavg = np.mean(errors[26:36])

print "No transform: ", noneavg
print "Translation: ", transavg
print "Scaling: ", scaleavg
print "Rotation: ", rotavg
print "Non linear: ", nlinavg

The above code computes the average error for each simulation type, shown below:

No transform:  0.0  
Translation:  7.03717370471  
Scaling:  2.10241035455  
Rotation:  1.50942092533  
Non linear:  1680.5

As we can see, the algorithm performs very well on the linear transformations and very poorly on the non-linear transformations.

#### Real data

Now we show the results of the algorithm on real data. We have two datasets: BNU2 and SWU4. Our reference is the MNI152_2mm brain scan. Below, we show the result of registering one brain from each dataset to the referecen.

The left images are the reference, the middle images are the preprocessed input brain, and the right images are the registered brain.

BNU2:

![real data](flirt_pics/realdata.jpg)

SWU4:

![real data](flirt_pics/realdata2.jpg)

Based on the above images, we can see that performing FLIRT does indeed make the input brains appear similar to the MNI reference brain. In order to evaluate the usefulness of the algorithm, we will have to quantify its performance on real data. We can do this by seeing if perform FLIRT improves dataset-wide discriminability: